# API de DataFrames en Profundidad

## Objetivos de Aprendizaje
- Dominar las operaciones avanzadas con DataFrames
- Trabajar con diferentes tipos de datos y esquemas
- Usar funciones built-in de Spark para transformaciones
- Manejar valores nulos y datos faltantes

## Prerequisitos
- `00_setup/02_spark_basics.ipynb`
- `06_spark_processing/01_rdd_fundamentals.ipynb`

## Tiempo Estimado
⏱️ 60 minutos

## Módulo AWS Academy Relacionado
📚 Módulo 9: Big Data Processing - DataFrames y procesamiento estructurado

In [ ]:
# Imports necesarios
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime, date

# Crear sesion de Spark
spark = SparkSession.builder \
    .appName("DataFrames_API") \
    .master("local[*]") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
print(f"Spark {spark.version} inicializado")

---
# === SECCIÓN 1 ===
## 1. Creación de DataFrames con Esquemas

### Explicación Conceptual
Un **esquema** define la estructura de un DataFrame: nombres de columnas, tipos de datos, y si permiten nulos. Definir el esquema explícitamente es más eficiente que dejar que Spark lo infiera.

**Analogía:** El esquema es como el plano de una casa. Define cuántas habitaciones hay (columnas), qué tipo de habitación es cada una (tipo de dato), y si son opcionales (nullable).

In [ ]:
# Definir un esquema complejo
# StructType es una lista de campos (columnas)
# StructField define: nombre, tipo, nullable

esquema_pedidos = StructType([
    StructField("pedido_id", StringType(), False),      # ID no puede ser nulo
    StructField("cliente_id", StringType(), False),
    StructField("fecha_pedido", DateType(), True),
    StructField("producto", StringType(), True),
    StructField("cantidad", IntegerType(), True),
    StructField("precio_unitario", DoubleType(), True),
    StructField("descuento", DoubleType(), True),       # Puede ser nulo
    StructField("enviado", BooleanType(), True)
])

# Datos de ejemplo (incluyendo valores nulos)
datos_pedidos = [
    ("P001", "C001", date(2024, 1, 15), "Laptop", 2, 15000.0, 0.10, True),
    ("P002", "C002", date(2024, 1, 16), "Mouse", 5, 350.0, None, True),
    ("P003", "C001", date(2024, 1, 17), "Monitor", 1, 8000.0, 0.05, False),
    ("P004", "C003", date(2024, 1, 18), "Teclado", 3, 800.0, None, True),
    ("P005", "C002", date(2024, 1, 19), "Laptop", 1, 15000.0, 0.15, False),
    ("P006", "C004", None, "Silla", 4, 2500.0, 0.0, None),  # Fecha nula
    ("P007", "C001", date(2024, 1, 20), None, 2, 500.0, None, True)  # Producto nulo
]

# Crear DataFrame con esquema explicito
df_pedidos = spark.createDataFrame(datos_pedidos, esquema_pedidos)

print("DataFrame de pedidos:")
df_pedidos.show()
df_pedidos.printSchema()

---
# === SECCIÓN 2 ===
## 2. Selección y Proyección de Columnas

### Explicación Conceptual
Seleccionar columnas es como elegir qué información necesitas de una tabla. Hay múltiples formas de hacerlo en Spark.

In [ ]:
# Diferentes formas de seleccionar columnas

# Forma 1: Por nombre (strings)
df_pedidos.select("pedido_id", "producto", "cantidad").show(3)

# Forma 2: Usando col()
df_pedidos.select(F.col("pedido_id"), F.col("producto")).show(3)

# Forma 3: Usando notacion de corchetes
df_pedidos.select(df_pedidos["pedido_id"], df_pedidos["producto"]).show(3)

# Forma 4: Seleccionar con expresiones calculadas
df_pedidos.select(
    F.col("producto"),
    F.col("cantidad"),
    F.col("precio_unitario"),
    (F.col("cantidad") * F.col("precio_unitario")).alias("subtotal")
).show()

In [ ]:
# Seleccionar todas las columnas excepto algunas
# drop() elimina columnas especificas

df_sin_descuento = df_pedidos.drop("descuento", "enviado")
print("Sin descuento y enviado:")
df_sin_descuento.show(3)

# Seleccionar columnas que cumplan un patron
# Ejemplo: todas las columnas que empiecen con 'precio'
columnas_precio = [c for c in df_pedidos.columns if 'precio' in c.lower()]
print(f"Columnas con 'precio': {columnas_precio}")

---
# === SECCIÓN 3 ===
## 3. Filtrado Avanzado

### Explicación Conceptual
Filtrar es seleccionar filas que cumplen ciertas condiciones. Spark soporta condiciones complejas con operadores lógicos.

In [ ]:
# Filtrado basico
print("Pedidos con cantidad > 2:")
df_pedidos.filter(F.col("cantidad") > 2).show()

# Filtro con multiples condiciones (AND)
print("Laptops enviadas:")
df_pedidos.filter(
    (F.col("producto") == "Laptop") & 
    (F.col("enviado") == True)
).show()

# Filtro con OR
print("Laptop o Monitor:")
df_pedidos.filter(
    (F.col("producto") == "Laptop") | 
    (F.col("producto") == "Monitor")
).show()

In [ ]:
# Filtros con funciones especiales

# isin(): Esta en una lista de valores
print("Productos en lista:")
df_pedidos.filter(F.col("producto").isin(["Laptop", "Mouse", "Teclado"])).show()

# like(): Patron SQL (% = cualquier cosa)
print("Productos que empiezan con 'M':")
df_pedidos.filter(F.col("producto").like("M%")).show()

# contains(): Contiene un substring
print("Productos que contienen 'o':")
df_pedidos.filter(F.col("producto").contains("o")).show()

# between(): Rango de valores
print("Precio entre 500 y 5000:")
df_pedidos.filter(F.col("precio_unitario").between(500, 5000)).show()

In [ ]:
# Filtrar valores nulos

# isNull(): Es nulo
print("Pedidos sin descuento (nulo):")
df_pedidos.filter(F.col("descuento").isNull()).show()

# isNotNull(): No es nulo
print("Pedidos con descuento:")
df_pedidos.filter(F.col("descuento").isNotNull()).show()

# Filas con CUALQUIER valor nulo
print("Pedidos con algun valor nulo:")
from functools import reduce
condicion_nulos = reduce(
    lambda a, b: a | b,
    [F.col(c).isNull() for c in df_pedidos.columns]
)
df_pedidos.filter(condicion_nulos).show()

---
# === SECCIÓN 4 ===
## 4. Agregar y Modificar Columnas

### Explicación Conceptual
`withColumn()` permite agregar nuevas columnas o modificar existentes. Las transformaciones son inmutables: siempre crean un nuevo DataFrame.

In [ ]:
# Agregar columnas calculadas
df_con_calculos = df_pedidos \
    .withColumn("subtotal", F.col("cantidad") * F.col("precio_unitario")) \
    .withColumn("descuento_monto", 
                F.when(F.col("descuento").isNotNull(), 
                       F.col("subtotal") * F.col("descuento"))
                .otherwise(0)) \
    .withColumn("total", F.col("subtotal") - F.col("descuento_monto"))

df_con_calculos.select(
    "pedido_id", "producto", "subtotal", "descuento_monto", "total"
).show()

In [ ]:
# Columnas condicionales con when/otherwise
# Es como un IF-THEN-ELSE

df_categorizado = df_pedidos.withColumn(
    "categoria_precio",
    F.when(F.col("precio_unitario") >= 10000, "Premium")
    .when(F.col("precio_unitario") >= 1000, "Medio")
    .otherwise("Economico")
)

df_categorizado.select("producto", "precio_unitario", "categoria_precio").show()

In [ ]:
# Renombrar columnas

# withColumnRenamed(): Renombra una columna
df_renombrado = df_pedidos.withColumnRenamed("precio_unitario", "precio")
print("Columnas renombradas:")
print(df_renombrado.columns)

# Renombrar multiples columnas con alias en select
df_pedidos.select(
    F.col("pedido_id").alias("id"),
    F.col("producto").alias("item"),
    F.col("precio_unitario").alias("precio")
).show(3)

---
# === SECCIÓN 5 ===
## 5. Funciones de Cadenas de Texto

### Explicación Conceptual
Spark incluye muchas funciones para manipular strings, similares a las funciones de SQL o Python.

In [ ]:
# Crear DataFrame de ejemplo con strings
df_strings = spark.createDataFrame([
    ("  Ana Garcia  ", "ana.garcia@email.com"),
    ("CARLOS LOPEZ", "Carlos.Lopez@EMPRESA.COM"),
    ("maria rodriguez", "MARIA@test.com")
], ["nombre", "email"])

df_strings.show()

In [ ]:
# Funciones de strings comunes

df_limpio = df_strings.select(
    F.col("nombre"),
    F.trim(F.col("nombre")).alias("nombre_trim"),           # Eliminar espacios
    F.lower(F.col("nombre")).alias("nombre_lower"),         # Minusculas
    F.upper(F.col("nombre")).alias("nombre_upper"),         # Mayusculas
    F.initcap(F.col("nombre")).alias("nombre_title"),       # Primera letra mayuscula
    F.length(F.trim(F.col("nombre"))).alias("longitud")     # Longitud
)

df_limpio.show(truncate=False)

In [ ]:
# Extraer partes de strings

df_email = df_strings.select(
    F.col("email"),
    # split() divide por un delimitador y retorna un array
    F.split(F.col("email"), "@").alias("partes"),
    # Acceder a elementos del array con getItem()
    F.split(F.col("email"), "@").getItem(0).alias("usuario"),
    F.split(F.col("email"), "@").getItem(1).alias("dominio"),
    # substring(col, inicio, longitud) - inicio es 1-based
    F.substring(F.col("email"), 1, 3).alias("primeros_3")
)

df_email.show(truncate=False)

In [ ]:
# Reemplazar y concatenar

df_transformado = df_strings.select(
    F.col("nombre"),
    # regexp_replace(col, patron, reemplazo)
    F.regexp_replace(F.col("nombre"), "\\s+", "_").alias("con_underscore"),
    # concat() une strings
    F.concat(F.col("nombre"), F.lit(" - "), F.col("email")).alias("combinado"),
    # concat_ws() une con separador
    F.concat_ws(" | ", F.col("nombre"), F.col("email")).alias("con_separador")
)

df_transformado.show(truncate=False)

---
# === SECCIÓN 6 ===
## 6. Funciones de Fecha y Hora

### Explicación Conceptual
Las fechas y horas son fundamentales en análisis de datos. Spark tiene funciones robustas para manipularlas.

In [ ]:
# Funciones de fecha

df_fechas = df_pedidos.filter(F.col("fecha_pedido").isNotNull()).select(
    F.col("pedido_id"),
    F.col("fecha_pedido"),
    # Extraer componentes de fecha
    F.year(F.col("fecha_pedido")).alias("anio"),
    F.month(F.col("fecha_pedido")).alias("mes"),
    F.dayofmonth(F.col("fecha_pedido")).alias("dia"),
    F.dayofweek(F.col("fecha_pedido")).alias("dia_semana"),  # 1=Domingo
    F.dayofyear(F.col("fecha_pedido")).alias("dia_anio"),
    F.weekofyear(F.col("fecha_pedido")).alias("semana"),
    F.quarter(F.col("fecha_pedido")).alias("trimestre")
)

df_fechas.show()

In [ ]:
# Operaciones con fechas

df_operaciones = df_pedidos.filter(F.col("fecha_pedido").isNotNull()).select(
    F.col("pedido_id"),
    F.col("fecha_pedido"),
    # Fecha actual
    F.current_date().alias("hoy"),
    # Diferencia en dias
    F.datediff(F.current_date(), F.col("fecha_pedido")).alias("dias_transcurridos"),
    # Sumar/restar dias
    F.date_add(F.col("fecha_pedido"), 30).alias("fecha_mas_30"),
    F.date_sub(F.col("fecha_pedido"), 7).alias("fecha_menos_7"),
    # Diferencia en meses
    F.months_between(F.current_date(), F.col("fecha_pedido")).alias("meses_transcurridos")
)

df_operaciones.show()

In [ ]:
# Formateo de fechas

df_formato = df_pedidos.filter(F.col("fecha_pedido").isNotNull()).select(
    F.col("fecha_pedido"),
    # date_format() convierte fecha a string con formato
    F.date_format(F.col("fecha_pedido"), "dd/MM/yyyy").alias("formato_mx"),
    F.date_format(F.col("fecha_pedido"), "MMMM dd, yyyy").alias("formato_largo"),
    F.date_format(F.col("fecha_pedido"), "EEEE").alias("nombre_dia")
)

df_formato.show(truncate=False)

---
# === SECCIÓN 7 ===
## 7. Manejo de Valores Nulos

### Explicación Conceptual
Los datos del mundo real contienen valores faltantes. Spark ofrece varias estrategias para manejarlos.

In [ ]:
# Ver donde hay nulos
print("Conteo de nulos por columna:")
df_pedidos.select([
    F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in df_pedidos.columns
]).show()

In [ ]:
# Eliminar filas con nulos

# Eliminar filas con CUALQUIER nulo
df_sin_nulos = df_pedidos.dropna()
print(f"Original: {df_pedidos.count()} filas")
print(f"Sin ningun nulo: {df_sin_nulos.count()} filas")

# Eliminar filas con nulo en columnas especificas
df_sin_nulos_producto = df_pedidos.dropna(subset=["producto", "fecha_pedido"])
print(f"Sin nulo en producto/fecha: {df_sin_nulos_producto.count()} filas")

# Eliminar solo si hay minimo N nulos
df_con_algunos = df_pedidos.dropna(thresh=6)  # Mantener si hay al menos 6 no-nulos
print(f"Con al menos 6 valores: {df_con_algunos.count()} filas")

In [ ]:
# Rellenar nulos

# Rellenar con valor especifico
df_rellenado = df_pedidos.fillna({
    "descuento": 0.0,
    "enviado": False,
    "producto": "Sin especificar"
})

print("Con nulos rellenados:")
df_rellenado.show()

In [ ]:
# Rellenar con coalesce() - primer valor no nulo

df_coalesce = df_pedidos.withColumn(
    "descuento_final",
    F.coalesce(F.col("descuento"), F.lit(0.0))
)

df_coalesce.select("pedido_id", "descuento", "descuento_final").show()

---
# === EJERCICIOS PRÁCTICOS ===

### 🎯 Ejercicio DF.1: Transformación Completa

Dado el DataFrame de pedidos:
1. Filtra pedidos del cliente "C001"
2. Calcula el total (cantidad × precio - descuento)
3. Agrega una columna "urgente" que sea True si el pedido no ha sido enviado
4. Ordena por total descendente

**Pistas:**
- Usa `coalesce()` para manejar descuentos nulos
- Usa `when()` para la columna urgente

In [ ]:
# TODO: Completa el ejercicio


### ✅ Solución Ejercicio DF.1

In [ ]:
# Solucion
resultado = df_pedidos \
    .filter(F.col("cliente_id") == "C001") \
    .withColumn("subtotal", F.col("cantidad") * F.col("precio_unitario")) \
    .withColumn("descuento_valor", 
                F.col("subtotal") * F.coalesce(F.col("descuento"), F.lit(0.0))) \
    .withColumn("total", F.col("subtotal") - F.col("descuento_valor")) \
    .withColumn("urgente", 
                F.when(F.col("enviado") == False, True)
                .when(F.col("enviado").isNull(), True)
                .otherwise(False)) \
    .orderBy(F.col("total").desc())

resultado.select(
    "pedido_id", "producto", "total", "enviado", "urgente"
).show()

### 🎯 Ejercicio DF.2: Análisis de Fechas

1. Agrega columna con el día de la semana en español
2. Agrega columna indicando si es fin de semana
3. Muestra cuántos pedidos hay por día de la semana

**Pistas:**
- `dayofweek()` retorna 1=Domingo, 2=Lunes, etc.
- Usa `when()` encadenado para mapear números a nombres

In [ ]:
# TODO: Completa el ejercicio


### ✅ Solución Ejercicio DF.2

In [ ]:
# Solucion
df_dias = df_pedidos \
    .filter(F.col("fecha_pedido").isNotNull()) \
    .withColumn("dia_num", F.dayofweek(F.col("fecha_pedido"))) \
    .withColumn("dia_nombre",
        F.when(F.col("dia_num") == 1, "Domingo")
        .when(F.col("dia_num") == 2, "Lunes")
        .when(F.col("dia_num") == 3, "Martes")
        .when(F.col("dia_num") == 4, "Miercoles")
        .when(F.col("dia_num") == 5, "Jueves")
        .when(F.col("dia_num") == 6, "Viernes")
        .when(F.col("dia_num") == 7, "Sabado")
    ) \
    .withColumn("es_fin_semana",
        F.col("dia_num").isin([1, 7])
    )

print("Pedidos con dia de la semana:")
df_dias.select("pedido_id", "fecha_pedido", "dia_nombre", "es_fin_semana").show()

print("Pedidos por dia:")
df_dias.groupBy("dia_nombre").count().orderBy("count", ascending=False).show()

### 🎯 Ejercicio DF.3: Limpieza de Datos

1. Identifica y muestra las filas con valores nulos
2. Crea una versión "limpia" del DataFrame:
   - Rellena descuentos nulos con 0
   - Rellena productos nulos con "DESCONOCIDO"
   - Elimina filas sin fecha
3. Verifica que no hay nulos en la versión limpia

In [ ]:
# TODO: Completa el ejercicio


### ✅ Solución Ejercicio DF.3

In [ ]:
# Solucion

# 1. Mostrar filas con nulos
print("Filas con al menos un valor nulo:")
condicion = reduce(
    lambda a, b: a | b,
    [F.col(c).isNull() for c in df_pedidos.columns]
)
df_pedidos.filter(condicion).show()

# 2. Crear version limpia
df_limpio = df_pedidos \
    .fillna({"descuento": 0.0, "producto": "DESCONOCIDO", "enviado": False}) \
    .dropna(subset=["fecha_pedido"])

print("\nDataFrame limpio:")
df_limpio.show()

# 3. Verificar que no hay nulos
print("\nConteo de nulos en DataFrame limpio:")
df_limpio.select([
    F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in df_limpio.columns
]).show()

---
# === RESUMEN FINAL ===

## Resumen

### Conceptos Clave
- **Esquema**: Define estructura (tipos y nullable) - mejor explícito que inferido
- **Selección**: `select()`, `drop()` para elegir columnas
- **Filtrado**: `filter()` con condiciones, `isin()`, `like()`, `isNull()`
- **Transformación**: `withColumn()`, `when/otherwise` para condicionales
- **Strings**: `trim()`, `lower()`, `split()`, `regexp_replace()`
- **Fechas**: `year()`, `month()`, `datediff()`, `date_format()`
- **Nulos**: `dropna()`, `fillna()`, `coalesce()`

### Conexión con AWS
- **AWS Glue**: Usa DynamicFrames con operaciones similares
- **Athena**: Funciones SQL equivalentes sobre datos en S3
- **EMR**: Ejecuta este mismo código en clusters grandes

### Siguiente Paso
Continúa con: `03_spark_sql.ipynb` para dominar SQL en Spark